# **CBD APPEND ID TASK**

---

# **DEBIT**

In [9]:
import pandas as pd
import numpy as np

init_df = pd.read_csv("CBD_FILES/CBD DEBIT INIT.csv", dtype={'TXN_REF_NB': str})
output_df = pd.read_csv("CBD_FILES/CBD DEBIT OUTPUT.csv")

init_df['TXN_REF_NB'] = init_df['TXN_REF_NB'].str.zfill(12)

init_df_renamed = init_df.rename(columns={
    'ISSU_CRCY_EQ_AM': 'Amount',
    'MRCH_TY_CD': 'Mcc',
    'ACPTR_NM': 'Raw description'
})

init_df_unique = init_df_renamed.drop_duplicates(subset=['Amount', 'Mcc', 'Raw description']).copy()

init_df_unique['Amount'] = init_df_unique['Amount'].astype(float)
output_df['Amount'] = output_df['Amount'].astype(float)

def fuzzy_merge_unique(df1, df2, on, tolerance):
    merged = []
    for idx, row in df2.iterrows():
        matches = df1[
            (np.abs(df1[on] - row[on]) <= tolerance) &
            (df1['Mcc'] == row['Mcc']) &
            (df1['Raw description'] == row['Raw description'])
        ]
        if not matches.empty:
            matches = matches.copy()
            matches['diff'] = np.abs(matches[on] - row[on])
            match_row = matches.sort_values('diff').iloc[0]
            merged.append({**row, 'TXN_REF_NB': match_row['TXN_REF_NB']})
        else:
            merged.append({**row, 'TXN_REF_NB': None})
    return pd.DataFrame(merged)

merged_df = fuzzy_merge_unique(init_df_unique, output_df, 'Amount', tolerance=1)

cols = ['TXN_REF_NB'] + [col for col in merged_df.columns if col != 'TXN_REF_NB']
merged_df = merged_df[cols]

merged_df.to_excel("CBD_FILES/CBD DEBIT OUTPUT FINAL FIX.xlsx", index=False)

print("✅ Done: TXN_REF_NB uniquely matched using fuzzy matching with tolerance, moved to front, and saved correctly.")

✅ Done: TXN_REF_NB uniquely matched using fuzzy matching with tolerance, moved to front, and saved correctly.


---

# **ATM**

In [ ]:
import pandas as pd
import numpy as np

init_df = pd.read_csv("CBD_FILES/CBD ATM INIT.csv", dtype={'TXN_REF_NB': str})
output_df = pd.read_csv("CBD_FILES/CBD ATM OUTPUT NEW.csv")

init_df['TXN_REF_NB'] = init_df['TXN_REF_NB'].str.zfill(12)

init_df_renamed = init_df.rename(columns={
    'TXN_AM': 'Amount',
    'MRCH_TY_CD': 'Mcc',
    'ACPTR_NM': 'Raw Description'
})

init_df_unique = init_df_renamed.drop_duplicates(subset=['Amount', 'Mcc', 'Raw Description']).copy()

init_df_unique['Amount'] = init_df_unique['Amount'].astype(float)
output_df['Amount'] = output_df['Amount'].astype(float)

def fuzzy_merge_unique(df1, df2, on, tolerance):
    merged = []
    for idx, row in df2.iterrows():
        matches = df1[
            (np.abs(df1[on] - row[on]) <= tolerance) &
            (df1['Mcc'] == row['Mcc']) &
            (df1['Raw Description'] == row['Raw Description']) 
        ]
        if not matches.empty:
            matches = matches.copy()
            matches['diff'] = np.abs(matches[on] - row[on])
            match_row = matches.sort_values('diff').iloc[0]
            merged.append({**row, 'TXN_REF_NB': match_row['TXN_REF_NB']})
        else:
            merged.append({**row, 'TXN_REF_NB': None})
    return pd.DataFrame(merged)

merged_df = fuzzy_merge_unique(init_df_unique, output_df, 'Amount', tolerance=1)

cols = ['TXN_REF_NB'] + [col for col in merged_df.columns if col != 'TXN_REF_NB']
merged_df = merged_df[cols]

merged_df.to_excel("CBD_FILES/CBD ATM OUTPUT FINAL FIX.xlsx", index=False)

print("✅ Done: TXN_REF_NB uniquely matched using fuzzy matching with tolerance, moved to front, and saved correctly.")

✅ Done: TXN_REF_NB uniquely matched using fuzzy matching with tolerance, moved to front, and saved correctly.


In [17]:
init_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   DP_history_acct_type            5000 non-null   object 
 1   DP_history_origin_ID            5000 non-null   int64  
 2   DP_history_origin_description   5000 non-null   object 
 3   DP_history_origin_branch_no     4996 non-null   float64
 4   DP_history_Transfer_acct_no     839 non-null    object 
 5   DP_history_Transfer_acct_type   839 non-null    object 
 6   DP_history_charge_desc          0 non-null      float64
 7   DP_history_atm_location         4096 non-null   object 
 8   DP_history_debit_credit         5000 non-null   object 
 9   DP_history_PTID                 5000 non-null   object 
 10  DP_history_parent_ptid          0 non-null      float64
 11  DP_history_Transaction_Desc     4957 non-null   object 
 12  DP_history_Reversal_PTID        18

In [15]:
output_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4957 entries, 0 to 4956
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               4957 non-null   int64  
 1   Raw description  4957 non-null   object 
 2   Category         4957 non-null   object 
 3   Sub category     4957 non-null   object 
 4   Brand            689 non-null    object 
 5   Brand logo       668 non-null    object 
 6   Mcc              4957 non-null   object 
 7   Amount           4957 non-null   float64
dtypes: float64(1), int64(1), object(6)
memory usage: 309.9+ KB


---

# **CC (credit)**

In [11]:
import pandas as pd
import numpy as np

init_df = pd.read_csv("CBD_FILES/CBD CC INIT.csv", dtype={'REF_NB': str})
output_df = pd.read_csv("CBD_FILES/CBD CC OUTPUT.csv")

# init_df['REF_NB'] = init_df['REF_NB'].str.zfill(12)

init_df_renamed = init_df.rename(columns={
    'TXN_AM': 'Amount',
    'MRCH_CAT_CD': 'Mcc',
    'TXN_DESC_TX': 'Raw description'
})

init_df_unique = init_df_renamed.drop_duplicates(subset=['Amount', 'Mcc', 'Raw description']).copy()

init_df_unique['Amount'] = init_df_unique['Amount'].astype(float)
output_df['Amount'] = output_df['Amount'].astype(float)

def fuzzy_merge_unique(df1, df2, on, tolerance):
    merged = []
    for idx, row in df2.iterrows():
        matches = df1[
            (np.abs(df1[on] - row[on]) <= tolerance) &
            (df1['Mcc'] == row['Mcc']) &
            (df1['Raw description'] == row['Raw description'])
        ]
        if not matches.empty:
            matches = matches.copy()
            matches['diff'] = np.abs(matches[on] - row[on])
            match_row = matches.sort_values('diff').iloc[0]
            merged.append({**row, 'REF_NB': match_row['REF_NB']})
        else:
            merged.append({**row, 'REF_NB': None})
    return pd.DataFrame(merged)

merged_df = fuzzy_merge_unique(init_df_unique, output_df, 'Amount', tolerance=1)

cols = ['REF_NB'] + [col for col in merged_df.columns if col != 'REF_NB']
merged_df = merged_df[cols]

merged_df.to_excel("CBD_FILES/CBD CC OUTPUT FINAL FIX.xlsx", index=False)

print("✅ Done: REF_NB uniquely matched using fuzzy matching with tolerance, moved to front, and saved correctly.")

✅ Done: REF_NB uniquely matched using fuzzy matching with tolerance, moved to front, and saved correctly.


---

# **DR**

In [12]:
import pandas as pd
import numpy as np

init_df = pd.read_csv("CBD_FILES/CBD ACC DR INIT.csv", dtype={'DP_history_PTID': str})
output_df = pd.read_csv("CBD_FILES/CBD ACC DR OUTPUT.csv")

init_df_renamed = init_df.rename(columns={
    'DP_history_amount_FCY': 'Amount',
    'DP_history_Transaction_Desc': 'Raw description'
})

init_df_unique = init_df_renamed.drop_duplicates(subset=['Amount', 'Raw description']).copy()

init_df_unique['Amount'] = init_df_unique['Amount'].astype(float)
output_df['Amount'] = output_df['Amount'].astype(float)

def fuzzy_merge_unique(df1, df2, on, tolerance):
    merged = []
    for idx, row in df2.iterrows():
        matches = df1[
            (np.abs(df1[on] - row[on]) <= tolerance) &
            (df1['Raw description'] == row['Raw description'])
        ]
        if not matches.empty:
            matches = matches.copy()
            matches['diff'] = np.abs(matches[on] - row[on])
            match_row = matches.sort_values('diff').iloc[0]
            merged.append({**row, 'DP_history_PTID': match_row['DP_history_PTID']})
        else:
            merged.append({**row, 'DP_history_PTID': None})
    return pd.DataFrame(merged)

merged_df = fuzzy_merge_unique(init_df_unique, output_df, 'Amount', tolerance=1)

cols = ['DP_history_PTID'] + [col for col in merged_df.columns if col != 'DP_history_PTID']
merged_df = merged_df[cols]

merged_df.to_excel("CBD_FILES/CBD ACC DR OUTPUT FINAL FIX.xlsx", index=False)

print("✅ Done: DP_history_PTID uniquely matched using fuzzy matching with tolerance, moved to front, and saved correctly.")

✅ Done: DP_history_PTID uniquely matched using fuzzy matching with tolerance, moved to front, and saved correctly.


---

# **CR**

In [13]:
import pandas as pd
import numpy as np

init_df = pd.read_csv("CBD_FILES/CBD ACC CR INIT.csv", dtype={'DP_history_PTID': str})
output_df = pd.read_csv("CBD_FILES/CBD ACC CR OUTPUT.csv")

init_df_renamed = init_df.rename(columns={
    'DP_history_amount_FCY': 'Amount',
    'DP_history_Transaction_Desc': 'Raw description'
})

init_df_unique = init_df_renamed.drop_duplicates(subset=['Amount', 'Raw description']).copy()

init_df_unique['Amount'] = init_df_unique['Amount'].astype(float)
output_df['Amount'] = output_df['Amount'].astype(float)

def fuzzy_merge_unique(df1, df2, on, tolerance):
    merged = []
    for idx, row in df2.iterrows():
        matches = df1[
            (np.abs(df1[on] - row[on]) <= tolerance) &
            (df1['Raw description'] == row['Raw description'])
        ]
        if not matches.empty:
            matches = matches.copy()
            matches['diff'] = np.abs(matches[on] - row[on])
            match_row = matches.sort_values('diff').iloc[0]
            merged.append({**row, 'DP_history_PTID': match_row['DP_history_PTID']})
        else:
            merged.append({**row, 'DP_history_PTID': None})
    return pd.DataFrame(merged)

merged_df = fuzzy_merge_unique(init_df_unique, output_df, 'Amount', tolerance=1)

cols = ['DP_history_PTID'] + [col for col in merged_df.columns if col != 'DP_history_PTID']
merged_df = merged_df[cols]

merged_df.to_excel("CBD_FILES/CBD ACC CR OUTPUT FINAL FIX.xlsx", index=False)

print("✅ Done: DP_history_PTID uniquely matched using fuzzy matching with tolerance, moved to front, and saved correctly.")

✅ Done: DP_history_PTID uniquely matched using fuzzy matching with tolerance, moved to front, and saved correctly.
